In [1]:
import sys
import argparse
import os
import torch
from collections import OrderedDict
import math
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from scipy.interpolate import griddata
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.gridspec as gridspec
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torch.utils.data import TensorDataset, DataLoader
from torchvision import datasets
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch
import seaborn as sns
import pylab as py
import time
from pyDOE import lhs
import warnings
sys.path.insert(0, '../../../Scripts/')
from models_imperfect import Generator, Discriminator, Q_Net
from pig import *
# from ../Scripts/helper import *

warnings.filterwarnings('ignore')

np.random.seed(1234)

In [2]:
# CUDA support 
if torch.cuda.is_available():
    device = torch.device('cuda:7')
else:
    device = torch.device('cpu')

## Hyper-parameters

In [3]:
num_epochs = 10000
lambda_val = 1
lambda_q = 0.5
tr_frac = 0.4

#architecture for the models
d_hid_dim = 50 
d_num_layer = 2

g_hid_dim = 50
g_num_layer = 4

q_hid_dim = 50
q_num_layer = 3

### Load Data

In [4]:
data = np.loadtxt('../../../datasets/tossing_trajectories.txt')
x = data[:, 0:6]
labels = data[:, 6:]

# training and test split
n_obs = int(tr_frac * x.shape[0])
train_x , train_y = x[:n_obs,:] , labels[:n_obs, :] 
test_x , test_y = x[n_obs:,:] , labels[n_obs:, :]

noise_dim = 2
data_dim = train_x.shape[1]
out_dim = labels.shape[1]

#### Models

In [5]:
D = Discriminator(in_dim = (data_dim + out_dim), out_dim = 1, hid_dim=d_hid_dim, num_layers=d_num_layer).to(device)
G = Generator(in_dim = (noise_dim + data_dim), out_dim = out_dim, hid_dim=g_hid_dim, num_layers=g_num_layer).to(device)
Q = Q_Net(in_dim=(out_dim + data_dim), out_dim=noise_dim, hid_dim=q_hid_dim, num_layers=q_num_layer).to(device)

In [6]:
PIG = Tossing_PIG(train_x, train_y, test_x, test_y, G, D, Q, device, num_epochs, lambda_val, noise_dim)

### Training

In [7]:
PIG.train()

[Epoch 0/10000] [MSE loss: 3.962500] [G loss: 446.361282] [D loss: 5.473109] [Q loss: 4.020192] [Phy loss: 444.083237] [Adv G loss: 0.540978]
[Epoch 100/10000] [MSE loss: 0.240060] [G loss: 17.013950] [D loss: 5.197514] [Q loss: 3.390023] [Phy loss: 14.638809] [Adv G loss: 0.605100]
[Epoch 200/10000] [MSE loss: 0.088430] [G loss: 4.475567] [D loss: 5.285515] [Q loss: 3.870740] [Phy loss: 1.731760] [Adv G loss: 0.591088]
[Epoch 300/10000] [MSE loss: 0.189821] [G loss: 4.535903] [D loss: 5.144568] [Q loss: 4.220296] [Phy loss: 1.773377] [Adv G loss: 1.004580]
[Epoch 400/10000] [MSE loss: 0.100596] [G loss: 5.806224] [D loss: 4.837609] [Q loss: 4.285120] [Phy loss: 1.955797] [Adv G loss: 1.948232]
[Epoch 500/10000] [MSE loss: 0.028258] [G loss: 6.232815] [D loss: 5.001040] [Q loss: 4.401441] [Phy loss: 1.881438] [Adv G loss: 2.160344]
[Epoch 600/10000] [MSE loss: 0.049339] [G loss: 12.184770] [D loss: 3.384433] [Q loss: 4.154646] [Phy loss: 2.456382] [Adv G loss: 7.952081]
[Epoch 700/1000

In [8]:
nsamples = 500
train_predictions_list = []
for run in range(nsamples):
    G_train_noise = PIG.sample_noise(PIG.train_x.shape[0], PIG.noise_dim).to(device)
    train_predictions = PIG.G.forward(torch.cat([PIG.train_x, G_train_noise], dim=1)).detach().cpu().numpy()
    train_predictions = (train_predictions * PIG.Ystd) + PIG.Ymean
    train_predictions_list.append(train_predictions)

train_predictions = np.mean(train_predictions_list, axis=0)
train_predictions_dev = np.var(train_predictions_list, axis=0)
train_predictions = torch.Tensor(train_predictions).float().to(device)

std_y_t = torch.Tensor(PIG.Ystd).float().to(device)
mean_y_t = torch.Tensor(PIG.Ymean).float().to(device)
std_x_t = torch.Tensor(PIG.Xstd).float().to(device)
mean_x_t = torch.Tensor(PIG.Xmean).float().to(device)

train_y = (PIG.train_y * std_y_t) + mean_y_t
train_x = (PIG.train_x * std_x_t) + mean_x_t

mse_train = ((train_predictions - train_y)**2).mean()
phy_train = torch.mean(PIG.physics_loss(train_x, train_predictions, [0, 1], [0, 1])).detach().cpu().numpy()

print("MSE on Training Set : ", mse_train.detach().cpu().numpy())
print("Physical Consistency: ", phy_train)

MSE on Training Set :  0.14441592
Physical Consistency:  0.73300487


In [10]:
test_predictions_list = []
for run in range(nsamples):
    G_test_noise = PIG.sample_noise(PIG.test_x.shape[0], PIG.noise_dim).to(device)
    test_predictions = PIG.G.forward(torch.cat([PIG.test_x, G_test_noise], dim=1)).detach().cpu().numpy()
    test_predictions = (test_predictions * PIG.Ystd) + PIG.Ymean
    test_predictions_list.append(test_predictions)

test_predictions = np.mean(test_predictions_list, axis=0)
test_predictions_dev = np.var(test_predictions_list, axis=0)
test_predictions = torch.Tensor(test_predictions).float().to(device)

test_y = (PIG.test_y * std_y_t) + mean_y_t
test_x = (PIG.test_x * std_x_t) + mean_x_t

mse_test = ((test_predictions - test_y)**2).mean()
phy_test = torch.mean(PIG.physics_loss(test_x, test_predictions, [0, 1], [0, 1])).detach().cpu().numpy()

print("MSE on Test Set : ", mse_test.detach().cpu().numpy())
print("Physical Consistency: ", phy_test)

MSE on Test Set :  0.18706776
Physical Consistency:  0.45116368
